In [1]:
# !pip install transformers
# !pip install openai
# !pip install ipdb
# !pip install re
# !pip install tqdm
# !pip install conlleval
# !pip install wandb
# !pip install pandas --upgrade #restart runtime if on Google Colab
# !pip install torch==1.5.0

In [2]:
import transformers
import openai
import ipdb
import re
import tqdm
import conlleval
import wandb 
#restart runtime if on Google Colab
import torch
import pandas as pd

/usr/lib/python3/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [3]:
import os

In [4]:
os.environ["CUDA_VISIBLE_DEVICES"]="0"

In [5]:
def add_prompts(df):
    """
    Combining sentences and entities to create prompts in Dataframe with 'sents' and 'entities' columns.
    Adds 'prompts' and 'empty_prompts' (prompt without answer) columns to DataFrame
    """
    
    prompts = []
    empty_prompts = []
    labels = []
    
    for i, row in df.iterrows():
        sent = row['sentence']
        entity1 = row['entity1']
        entity2 = row['entity2']
        label = row['label'].split('-')[1]
        label = label_verbalizer[label]
        
        labels.append(label)
        prompt = prompt_sample_structure.format(sent, entity1, entity2, label)
        empty_prompt = empty_prompt_sample_structure.format(sent, entity1, entity2)
        
        prompts.append(prompt)
        empty_prompts.append(empty_prompt)

    df['prompts'] = prompts
    df['empty_prompts'] = empty_prompts
    df['labels'] = labels
    unique_labels = set(labels)
    df['unique_labels'] = [unique_labels for _ in empty_prompts]
    
    return df

In [6]:
verbalizer = 'natural'

#Basic Prompt Structure

if verbalizer == '':
    prompt_sample_structure = 'Sentence: {}\n'+entity_type+'1: {}\n'+entity_type+'2: {}\n'+relation_type+': {}'
    empty_prompt_sample_structure = 'Sentence: {}\n'+entity_type+'1: {}\n'+entity_type+'2: {}\n'+relation_type+':'

    label_verbalizer = {'effect':'effect',
                        'false':'none',
                        'advise':'advice',
                        'mechanism':'mechanism',
                        'int':'other'
                       }
elif verbalizer == 'natural':
    #More Natural Prompt Structure
    # How are entities {} and {} related based on the previous sentence? 
    prompt_sample_structure = 'Sentence: {}\nHow do drugs {} and {} interact based on the previous sentence?\nInteraction: {}'
    empty_prompt_sample_structure = 'Sentence: {}\nHow do  drugs {} and {} interact based on the previous sentence?\nInteraction:'

    label_verbalizer = {'effect':"Effect Alteration",
                        'false':'No Interaction',
                        'advise':'Recommendation',
                        'mechanism':'Metabolic Mechanism ',
                        'int':'Unspecified'
                       }
else:
    #More Natural Prompt Structure
    # How are entities {} and {} related based on the previous sentence? 
    prompt_sample_structure = 'Sentence: {}\nHow do drugs {} and {} interact based on the previous sentence?\n{}'
    empty_prompt_sample_structure = 'Sentence: {}\nHow do  drugs {} and {} interact based on the previous sentence?\n'

    label_verbalizer = {'effect':'They have an effect on each other.',
                        'false':'They have no interaction.',
                        'advise':'Some medical advice is given about their interaction.',
                        'mechanism':'They interact through their metabolic mechanisms.',
                        'int':'They have some unspecified interaction.'
    }

In [7]:
# train = pd.read_csv('train_set_balanced_50.csv') # 100
train = pd.read_csv('train_set_balanced_500_2_2.csv') # 500
# train = pd.read_csv('train_set_balanced_50.csv') # 50



# train = pd.read_csv('train_set_stratified_50_samples_seed_21.csv')



# train = pd.read_csv('ddi.train.tsv')
dev = pd.read_csv('ddi.dev.tsv')

In [8]:
len(train)

500

In [9]:
train = add_prompts(train)
dev = add_prompts(dev)

In [10]:
print(dev.prompts.values[1])

Sentence: In vitro mixing of an aminoglycoside with beta-lactamtype antibiotics (penicillins or cephalosporins) may result in a significant mutual inactivation.
How do drugs aminoglycoside and penicillins interact based on the previous sentence?
Interaction: Effect Alteration


In [11]:
def undersample(df, col_to_undersample, undersample_perc):
    
    new_df = []
    for label in label_order:
        
        g = df[df[col_to_undersample] == label_verbalizer[label]]
        print(label)
        if label == 'false':
            g = g[:int(len(g)*undersample_perc)]
        
        new_df.append(g)
        
    return pd.concat(new_df)

In [12]:
dev.groupby('labels').count()/len(dev)

,Unnamed: 0,index,sentence,label,doc,split,entity1,entity2,prompts,empty_prompts,unique_labels,test_ready_prompt
labels,,,,,,,,,,,,
Effect Alteration,0.243243,0.243243,0.243243,0.243243,0.243243,0.243243,0.243243,0.243243,0.243243,0.243243,0.243243,0.243243
Metabolic Mechanism,0.229115,0.229115,0.229115,0.229115,0.229115,0.229115,0.229115,0.229115,0.229115,0.229115,0.229115,0.229115
No Interaction,0.383292,0.383292,0.383292,0.383292,0.383292,0.383292,0.383292,0.383292,0.383292,0.383292,0.383292,0.383292
Recommendation,0.118550,0.118550,0.118550,0.118550,0.118550,0.118550,0.118550,0.118550,0.118550,0.118550,0.118550,0.118550
Unspecified,0.025799,0.025799,0.025799,0.025799,0.025799,0.025799,0.025799,0.025799,0.025799,0.025799,0.025799,0.025799


In [13]:
label_order = ['effect', 'false','advise','mechanism','int']
dev = undersample(dev, 'labels',0.1)
dev.groupby('labels').count()/len(dev)

effect
false
advise
mechanism
int


,Unnamed: 0,index,sentence,label,doc,split,entity1,entity2,prompts,empty_prompts,unique_labels,test_ready_prompt
labels,,,,,,,,,,,,
Effect Alteration,0.371482,0.371482,0.371482,0.371482,0.371482,0.371482,0.371482,0.371482,0.371482,0.371482,0.371482,0.371482
Metabolic Mechanism,0.349906,0.349906,0.349906,0.349906,0.349906,0.349906,0.349906,0.349906,0.349906,0.349906,0.349906,0.349906
No Interaction,0.058161,0.058161,0.058161,0.058161,0.058161,0.058161,0.058161,0.058161,0.058161,0.058161,0.058161,0.058161
Recommendation,0.181051,0.181051,0.181051,0.181051,0.181051,0.181051,0.181051,0.181051,0.181051,0.181051,0.181051,0.181051
Unspecified,0.039400,0.039400,0.039400,0.039400,0.039400,0.039400,0.039400,0.039400,0.039400,0.039400,0.039400,0.039400


In [14]:
train_filtered = train[["prompts", "entity1", "entity2", "labels"]]
dev_filtered = dev[["prompts", "entity1", "entity2", "labels"]]

# frames = [train_filtered, dev_filtered]
# full = pd.DataFrame(pd.concat(frames, ignore_index=True))
dev_filtered["prompts"] = pd.DataFrame(dev["prompts"].str.replace(r'Sentence: ',"").str.split('\n').str[0].astype(str).str[:-1])
train_filtered["prompts"] = pd.DataFrame(train["prompts"].str.replace(r'Sentence: ',"").str.split('\n').str[0].astype(str).str[:-1])

train_filtered = train_filtered.sort_values(by = 'labels')
dev_filtered = dev_filtered.sort_values(by = 'labels')
train_filtered.index=range(0,len(train_filtered))
dev_filtered.index=range(0,len(dev_filtered))
# dev_filtered = dev_filtered[dev_filtered['labels'].str.contains("Effect Alteration")]

# sentences = full['prompts']
train_filtered_labels = train_filtered['labels']
dev_filtered_labels = dev_filtered['labels']
train_filtered_labels.index=range(0,len(train_filtered_labels))
dev_filtered_labels.index=range(0,len(dev_filtered_labels))


# a temporary list to store the string labels
dev_labels_list = dev_filtered_labels.tolist()
dev_labels = pd.factorize(dev_labels_list)
# int_labels = int_labels[0]
dev_labels = dev_labels[0]


# a temporary list to store the string labels
train_labels_list = train_filtered_labels.tolist()
train_labels = pd.factorize(train_labels_list)
# int_labels = int_labels[0]
train_labels = train_labels[0]


dev_filtered.index=range(0,len(dev_filtered))

# for idx, element in enumerate(dev_labels):
#     if dev_filtered['labels'][idx] == 'Effect Alteration':
#         dev_labels[idx] = 0
#     if dev_filtered['labels'][idx] == 'No Interaction':
#         dev_labels[idx] = 1
#     if dev_filtered['labels'][idx] == 'Recommendation':
#         dev_labels[idx] = 2
#     if dev_filtered['labels'][idx] == 'Metabolic Mechanism ':
#         dev_labels[idx] = 3
#     if dev_filtered['labels'][idx] == 'Unspecified':
#         dev_labels[idx] = 4
    

/home/mcneal.121/.local/lib/python3.6/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/home/mcneal.121/.local/lib/python3.6/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys


In [15]:
dev_labels_list

['Effect Alteration',
 'Effect Alteration',
 'Effect Alteration',
 'Effect Alteration',
 'Effect Alteration',
 'Effect Alteration',
 'Effect Alteration',
 'Effect Alteration',
 'Effect Alteration',
 'Effect Alteration',
 'Effect Alteration',
 'Effect Alteration',
 'Effect Alteration',
 'Effect Alteration',
 'Effect Alteration',
 'Effect Alteration',
 'Effect Alteration',
 'Effect Alteration',
 'Effect Alteration',
 'Effect Alteration',
 'Effect Alteration',
 'Effect Alteration',
 'Effect Alteration',
 'Effect Alteration',
 'Effect Alteration',
 'Effect Alteration',
 'Effect Alteration',
 'Effect Alteration',
 'Effect Alteration',
 'Effect Alteration',
 'Effect Alteration',
 'Effect Alteration',
 'Effect Alteration',
 'Effect Alteration',
 'Effect Alteration',
 'Effect Alteration',
 'Effect Alteration',
 'Effect Alteration',
 'Effect Alteration',
 'Effect Alteration',
 'Effect Alteration',
 'Effect Alteration',
 'Effect Alteration',
 'Effect Alteration',
 'Effect Alteration',
 'Effect A

In [16]:
# tst = train_filtered['labels'] == 'Effect Alteration'
# tst = pd.DataFrame(train_labels)[0]
# train_filtered['labels'=="Effect Alteration"]

In [17]:
pd.set_option('display.max_rows', 1067)

In [18]:
# use dummy token "DRUG1" and "DRUG2"

for idx, sentence in enumerate(dev_filtered.transpose()):
  dev_filtered['prompts'][idx] = dev_filtered['prompts'][idx].replace(dev_filtered['entity1'][idx], "DRUG1")
  dev_filtered["prompts"][idx] = dev_filtered['prompts'][idx].replace(dev_filtered['entity2'][idx], "DRUG2")
  if "DRUG1s" in dev_filtered["prompts"][idx]:
    dev_filtered["prompts"][idx] = dev_filtered["prompts"][idx].replace("DRUG1s", "DRUG1")
  if "DRUG2s" in dev_filtered["prompts"][idx]:
    dev_filtered["prompts"][idx] = dev_filtered["prompts"][idx].replace("DRUG2s", "DRUG2")

sentences_dev = dev_filtered["prompts"]




for idx, sentence in enumerate(train_filtered.transpose()):
  train_filtered["prompts"][idx] = train_filtered['prompts'][idx].replace(train_filtered['entity1'][idx], "DRUG1")
  train_filtered["prompts"][idx] = train_filtered['prompts'][idx].replace(train_filtered['entity2'][idx], "DRUG2")
  if "DRUG1s" in train_filtered["prompts"][idx]:
    train_filtered["prompts"][idx] = train_filtered["prompts"][idx].replace("DRUG1s", "DRUG1")
  if "DRUG2s" in train_filtered["prompts"][idx]:
    train_filtered["prompts"][idx] = train_filtered["prompts"][idx].replace("DRUG2s", "DRUG2")

sentences_train = train_filtered["prompts"]


sentences_full = pd.merge(sentences_dev, sentences_train)

In [19]:
train_labels

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3,

In [20]:
print(sentences_train[0])
print(train_filtered['entity1'][0])
print(train_filtered['entity2'][0])
print(train_filtered['labels'][0])

The hypoglycemic action of DRUG1 may be potentiated by certain drugs including nonsteroidal anti-inflammatory agents and other drugs that are highly protein bound, DRUG2, sulfonamides, chloramphenicol, probenecid, coumarins, monoamine oxidase inhibitors, and beta adrenergic blocking agents
sulfonylurea
salicylates
Effect Alteration


In [21]:
import numpy as np

# Function to calculate the accuracy of our predictions vs labels
def flat_accuracy(preds, labels):
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return np.sum(pred_flat == labels_flat) / len(labels_flat)

In [22]:
from transformers import AutoTokenizer, AutoModelForMaskedLM

model_name = "microsoft/BiomedNLP-PubMedBERT-base-uncased-abstract"
tokenizer = AutoTokenizer.from_pretrained(model_name, do_lower_cased=True)
print(len(tokenizer))
special_tokens_dict = {'additional_special_tokens': ['DRUG1','DRUG2']}
num_added_toks = tokenizer.add_special_tokens(special_tokens_dict)
print(len(tokenizer))

max_len = 0
for sent in sentences_full:
  input_ids = tokenizer.encode(sent, add_special_tokens=True)
  max_len = max(max_len, len(input_ids))

28895
28897


In [23]:
epoch_count = 20
epoch_values = []
for i in range(epoch_count):
    epoch_values.append(i+1)

sweep_config = {
    'method': 'grid', #grid, random
    'metric': {
      'name': 'val_accuracy',
      'goal': 'maximize'   
    },
    'parameters': {

        'learning_rate': {
            'values': [ 1e-5, 3e-5, 5e-5] 
        },
        'batch_size': {
            'values': [16, 32] #16, 32
        },
        'epochs':{
            'values':[20]
        },
        'warmup_ratio':{
            'values': [0, 0.06]
        }
    }


    
}

sweep_defaults = {
    'learning_rate': 5e-5,
    'batch_size': 32,
    'epochs':50,
    'warmup_ratio':0
}

sweep_id = wandb.sweep(sweep_config)

Create sweep with ID: bm9f5y7r
Sweep URL: https://wandb.ai/nickmcneal/uncategorized/sweeps/bm9f5y7r


In [24]:
import torch
# Tokenize all of the sentences and map the tokens to thier word IDs.
input_ids_train = []
attention_masks_train = []

# For every sentence...
for sent in sentences_train:
    encoded_dict_train = tokenizer.encode_plus(
                        sent,                      # Sentence to encode.
                        add_special_tokens = True, # Add '[CLS]' and '[SEP]'
                        max_length = 64,           # Pad & truncate all sentences.
                        pad_to_max_length = True,
                        return_attention_mask = True,   # Construct attn. masks.
                        return_tensors = 'pt',     # Return pytorch tensors.
                   )
    
    # Add the encoded sentence to the list.    
    input_ids_train.append(encoded_dict_train['input_ids'])
    
    # And its attention mask (simply differentiates padding from non-padding).
    attention_masks_train.append(encoded_dict_train['attention_mask'])

# Convert the lists into tensors.
input_ids_train = torch.cat(input_ids_train, dim=0)
attention_masks_train = torch.cat(attention_masks_train, dim=0)
labels_train = torch.tensor(train_labels)

# Print sentence 0, now as a list of IDs.
print('Original: ', sentences_train[0])
print('Token IDs:', input_ids_train[0])



############################

input_ids_dev = []
attention_masks_dev = []

# For every sentence...
for sent in sentences_dev:
    encoded_dict_dev = tokenizer.encode_plus(
                        sent,                      # Sentence to encode.
                        add_special_tokens = True, # Add '[CLS]' and '[SEP]'
                        max_length = 64,           # Pad & truncate all sentences.
                        pad_to_max_length = True,
                        return_attention_mask = True,   # Construct attn. masks.
                        return_tensors = 'pt',     # Return pytorch tensors.
                   )
    
    # Add the encoded sentence to the list.    
    input_ids_dev.append(encoded_dict_dev['input_ids'])
    
    # And its attention mask (simply differentiates padding from non-padding).
    attention_masks_dev.append(encoded_dict_dev['attention_mask'])

# Convert the lists into tensors.
input_ids_dev = torch.cat(input_ids_dev, dim=0)
attention_masks_dev = torch.cat(attention_masks_dev, dim=0)
labels_dev = torch.tensor(dev_labels)

# Print sentence 0, now as a list of IDs.
print('Original: ', sentences_dev[0])
print('Token IDs:', input_ids_dev[0])

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/home/mcneal.121/.local/lib/python3.6/site-packages/transformers/tokenization_utils_base.py:2232: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


Original:  The hypoglycemic action of DRUG1 may be potentiated by certain drugs including nonsteroidal anti-inflammatory agents and other drugs that are highly protein bound, DRUG2, sulfonamides, chloramphenicol, probenecid, coumarins, monoamine oxidase inhibitors, and beta adrenergic blocking agents
Token IDs: tensor([    2,  1680, 21014,  3872,  1685, 28895,  2056,  1765, 14260,  1772,
         4823,  3524,  2574, 20008,  2000,    16,  3430,  3914,  1690,  2187,
         3524,  1760,  1810,  3343,  2031,  4426,    15, 28896,    15, 20024,
        16121,    15, 17307,    15,  5950,  2602,  9002,  1022,    15, 24622,
         1026,    15, 18269,  8182,  4299,    15,  1690,  2822,  9108,  8732,
         3914,     3,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0])
Original:  In vitro mixing of an DRUG1 with beta-lactamtype DRUG2 (penicillins or cephalosporins) may result in a significant mutual inactivation
Token IDs: tensor([    2,  1682,  30

In [25]:
labels_dev

tensor([0, 0, 0,  ..., 4, 4, 4])

In [26]:
from torch.utils.data import TensorDataset, random_split

# Combine the training inputs into a TensorDataset.
train_dataset = TensorDataset(input_ids_train, attention_masks_train, labels_train)
val_dataset = TensorDataset(input_ids_dev, attention_masks_dev, labels_dev)



# # Create a 90-10 train-validation split.

# # Calculate the number of samples to include in each set.
# train_size = int(0.9 * len(dataset))
# val_size = len(dataset) - train_size

# # Divide the dataset by randomly selecting samples.
# train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

# print('{:>5,} training samples'.format(train_size))
# print('{:>5,} validation samples'.format(val_size))

In [27]:
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler
import wandb
# WANDB PARAMETER
def ret_dataloader():
    batch_size = wandb.config.batch_size
    print('batch_size = ', batch_size)
    train_dataloader = DataLoader(
                train_dataset,  # The training samples.
                sampler = RandomSampler(train_dataset), # Select batches randomly
                batch_size = batch_size # Trains with this batch size.
            )

    validation_dataloader = DataLoader(
                val_dataset, # The validation samples.
                sampler = SequentialSampler(val_dataset), # Pull out batches sequentially.
                batch_size = batch_size # Evaluate with this batch size.
            )
    return train_dataloader,validation_dataloader

In [28]:
from transformers import AutoModelForSequenceClassification, AdamW, BertConfig

def ret_model():
    model_name = "microsoft/BiomedNLP-PubMedBERT-base-uncased-abstract"
    model = AutoModelForSequenceClassification.from_pretrained(model_name, 
        num_labels = 5, 
        output_attentions = False, # Whether the model returns attentions weights.
        output_hidden_states = False, # Whether the model returns all hidden-states.  
        )                             
    model.resize_token_embeddings(len(tokenizer))
    return model



In [29]:
def ret_optim(model):
    print('Learning_rate = ',wandb.config.learning_rate )
    optimizer = AdamW(model.parameters(),
                      lr = wandb.config.learning_rate, 
                      eps = 1e-8 
                    )
    return optimizer


In [30]:
from transformers import get_linear_schedule_with_warmup

def ret_scheduler(train_dataloader,optimizer):
    epochs = wandb.config.epochs
    warmup_ratio = wandb.config.warmup_ratio
#     ipdb.set_trace()
    print('epochs =>', epochs)
    # Total number of training steps is [number of batches] x [number of epochs]. 
    # (Note that this is not the same as the number of training samples).
    total_steps = len(train_dataloader) * epochs

    num_warmup_steps = warmup_ratio * total_steps
    # Create the learning rate scheduler.
    scheduler = get_linear_schedule_with_warmup(optimizer,
                                                num_warmup_steps = num_warmup_steps, # Default value in run_glue.py
                                                num_training_steps = total_steps)
    return scheduler

In [31]:
import numpy as np
from sklearn.metrics import precision_recall_fscore_support
# Function to calculate the accuracy of our predictions vs labels
def flatten(preds, labels):
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
#     precision_recall_fscore_support(y_true, y_pred, average='micro')


import time
import datetime

def format_time(elapsed):
    '''
    Takes a time in seconds and returns a string hh:mm:ss
    '''
    # Round to the nearest second.
    elapsed_rounded = int(round((elapsed)))
    
    # Format as hh:mm:ss
    return str(datetime.timedelta(seconds=elapsed_rounded))

In [32]:
import random
import numpy as np

    # This training code is based on the `run_glue.py` script here:
    # https://github.com/huggingface/transformers/blob/5bfcd0485ece086ebcbed2d008813037968a9e58/examples/run_glue.py#L128

    # Set the seed value all over the place to make this reproducible.
    
    

    
            
#         train_dataset = TensorDataset(input_ids_train, attention_masks_train, labels_train)
#         val_dataset = TensorDataset(input_ids_dev, attention_masks_dev, labels_dev)

    

data2 = pd.DataFrame(columns=['epoch', 'learning_rate', 'batch_size', 'warmup_ratio', 'predictions', 'labels', 'total_precision', 'total_recall', 'total_f1', 'categories_precision', 'categories_recall', 'categories_f1'])

def train():
    


    wandb.init(config=sweep_defaults)
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    print(device)
    model = ret_model()
    model.to(device)
    #wandb.init(config=sweep_defaults)
    train_dataloader,validation_dataloader = ret_dataloader()
    optimizer = ret_optim(model)
    scheduler = ret_scheduler(train_dataloader,optimizer)

    #print("config ",wandb.config.learning_rate, "\n",wandb.config)
    seed_val = 42

    random.seed(seed_val)
    np.random.seed(seed_val)
    torch.manual_seed(seed_val)
    #torch.cuda.manual_seed_all(seed_val)

    # We'll store a number of quantities such as training and validation loss, 
    # validation accuracy, and timings.
    training_stats = []

    # Measure the total training time for the whole run.
    total_t0 = time.time()
    epochs = wandb.config.epochs
    warmup_ratio = wandb.config.warmup_ratio
    # For each epoch...
    for epoch_i in range(0, epochs):

        # ========================================
        #               Training
        # ========================================

        # Perform one full pass over the training set.

        print("")
        print('======== Epoch {:} / {:} ========'.format(epoch_i + 1, epochs))
        print('Training...')

        # Measure how long the training epoch takes.
        t0 = time.time()

        # Reset the total loss for this epoch.
        total_train_loss = 0

        # Put the model into training mode. Don't be mislead--the call to 
        # `train` just changes the *mode*, it doesn't *perform* the training.
        # `dropout` and `batchnorm` layers behave differently during training
        # vs. test (source: https://stackoverflow.com/questions/51433378/what-does-model-train-do-in-pytorch)
        model.train()

        # For each batch of training data...
        for step, batch in enumerate(train_dataloader):

            # Progress update every 40 batches.
            if step % 40 == 0 and not step == 0:
                # Calculate elapsed time in minutes.
                elapsed = format_time(time.time() - t0)

                # Report progress.
                print('  Batch {:>5,}  of  {:>5,}.    Elapsed: {:}.'.format(step, len(train_dataloader), elapsed))

            # Unpack this training batch from our dataloader. 
            #
            # As we unpack the batch, we'll also copy each tensor to the GPU using the 
            # `to` method.
            #
            # `batch` contains three pytorch tensors:
            #   [0]: input ids 
            #   [1]: attention masks
            #   [2]: labels 
            b_input_ids = batch[0].to(device)
            b_input_mask = batch[1].to(device)
            b_labels = batch[2].to(device)
            # Always clear any previously calculated gradients before performing a
            # backward pass. PyTorch doesn't do this automatically because 
            # accumulating the gradients is "convenient while training RNNs". 
            # (source: https://stackoverflow.com/questions/48001598/why-do-we-need-to-call-zero-grad-in-pytorch)
            model.zero_grad()        

            # Perform a forward pass (evaluate the model on this training batch).
            # The documentation for this `model` function is here: 
            # https://huggingface.co/transformers/v2.2.0/model_doc/bert.html#transformers.BertForSequenceClassification
            # It returns different numbers of parameters depending on what arguments
            # arge given and what flags are set. For our useage here, it returns
            # the loss (because we provided labels) and the "logits"--the model
            # outputs prior to activation.
            outputs = model(b_input_ids, 
                                token_type_ids=None, 
                                attention_mask=b_input_mask, 
                                labels=b_labels)
            loss, logits = outputs['loss'], outputs['logits']
            wandb.log({'train_batch_loss':loss.item()})
            # Accumulate the training loss over all of the batches so that we can
            # calculate the average loss at the end. `loss` is a Tensor containing a
            # single value; the `.item()` function just returns the Python value 
            # from the tensor.
            total_train_loss += loss.item()

            # Perform a backward pass to calculate the gradients.
            loss.backward()

            # Clip the norm of the gradients to 1.0.
            # This is to help prevent the "exploding gradients" problem.
            torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

            # Update parameters and take a step using the computed gradient.
            # The optimizer dictates the "update rule"--how the parameters are
            # modified based on their gradients, the learning rate, etc.
            optimizer.step()

            # Update the learning rate.
            scheduler.step()

        # Calculate the average loss over all of the batches.
        avg_train_loss = total_train_loss / len(train_dataloader)            

        # Measure how long this epoch took.
        training_time = format_time(time.time() - t0)

        wandb.log({'avg_train_loss':avg_train_loss})

        print("")
        print("  Average training loss: {0:.2f}".format(avg_train_loss))
        print("  Training epoch took: {:}".format(training_time))

        # ========================================
        #               Validation
        # ========================================
        # After the completion of each training epoch, measure our performance on
        # our validation set.

        print("")
        print("Running Validation...")

        t0 = time.time()

        # Put the model in evaluation mode--the dropout layers behave differently
        # during evaluation.
        model.eval()

        # Tracking variables 
        total_eval_accuracy = 0
        total_eval_loss = 0
        nb_eval_steps = 0

        predictions_list = []
        labels_list = []
        # Evaluate data for one epoch
        for batch in validation_dataloader:

            # Unpack this training batch from our dataloader. 
            #
            # As we unpack the batch, we'll also copy each tensor to the GPU using 
            # the `to` method.
            #
            # `batch` contains three pytorch tensors:
            #   [0]: input ids 
            #   [1]: attention masks
            #   [2]: labels 
            b_input_ids = batch[0].cuda()
            b_input_mask = batch[1].to(device)
            b_labels = batch[2].to(device)

            # Tell pytorch not to bother with constructing the compute graph during
            # the forward pass, since this is only needed for backprop (training).
            with torch.no_grad():        

                # Forward pass, calculate logit predictions.
                # token_type_ids is the same as the "segment ids", which 
                # differentiates sentence 1 and 2 in 2-sentence tasks.
                # The documentation for this `model` function is here: 
                # https://huggingface.co/transformers/v2.2.0/model_doc/bert.html#transformers.BertForSequenceClassification
                # Get the "logits" output by the model. The "logits" are the output
                # values prior to applying an activation function like the softmax.
                outputs = model(b_input_ids, 
                                      token_type_ids=None, 
                                      attention_mask=b_input_mask,
                                      labels=b_labels)
                loss, logits = outputs['loss'], outputs['logits']
            # Accumulate the validation loss.
            total_eval_loss += loss.item()

            # Move logits and labels to CPU
            logits = logits.detach().cpu().numpy()
            label_ids = b_labels.to('cpu').numpy()

            # Calculate the accuracy for this batch of test sentences, and
            # accumulate it over all batches.
            total_eval_accuracy +=flat_accuracy(logits,label_ids)

            pred_flat = np.argmax(logits, axis=1).flatten()
            labels_flat = label_ids.flatten()
#             total_eval_accuracy = flatten(logits, label_ids)
            predictions_list.extend(pred_flat)
            labels_list.extend(labels_flat)
#             predictions_list = flatten(predictions_list)
#             labels_list = flatten(labels_list)
#             print("predictions: " + str(predictions_list))
#             print("labels: " + str(labels_list))

        # Report the final accuracy for this validation run.
        avg_val_accuracy = total_eval_accuracy / len(validation_dataloader)

#         print("  Accuracy: {0:.2f}".format(avg_val_accuracy))
#         ipdb.set_trace()
        # Calculate the average loss over all of the batches.
        avg_val_loss = total_eval_loss / len(validation_dataloader)
        wandb.log({'val_accuracy':avg_val_accuracy,'avg_val_loss':avg_val_loss})

        f1 = precision_recall_fscore_support(labels_list, predictions_list, average=None)
        p,r,f,s = precision_recall_fscore_support(y_pred=predictions_list, y_true=labels_list, labels=[0,1,3,4], average='micro')

        # Measure how long the validation run took.
        validation_time = format_time(time.time() - t0)
#         wandb.log({'val_f1':f1,'avg_val_loss':avg_val_loss})
        print("  Validation Loss: {0:.2f}".format(avg_val_loss))
        print("  Validation took: {:}".format(validation_time))
        print("labels: " + str(labels_list))
        print("predictions: " + str(predictions_list))
        print("  Total Precision: {:}".format(p))
        print("  Total Recall: {:}".format(r))
        print("  Total F1: {:}".format(f))
        print("  Categories Precision: {:}".format(f1[0]))
        print("  Categories Recall: {:}".format(f1[1]))
        print("  Categories F1: {:}".format(f1[2]))
#         print("  F1 score: {:}".format(f1))
        global data2
        data2 = data2.append({'epoch': epoch_i, 'learning_rate': wandb.config.learning_rate, 'batch_size': wandb.config.batch_size, 'warmup_ratio': wandb.config.warmup_ratio, 'predictions': str(predictions_list), 'labels': str(labels_list), 'total_precision': p, 'total_recall': r, 'total_f1': f, 'categories_precision': f1[0], 'categories_recall': f1[1], 'categories_f1': f1[2]}, ignore_index=True)

#             ipdb.set_trace()
#             print(data2)
        # Record all statistics from this epoch.
        training_stats.append(
            {
                'epoch': epoch_i + 1,
                'Training Loss': avg_train_loss,
                'Valid. Loss': avg_val_loss,
                'Valid. Accur.': avg_val_accuracy,
                'Training Time': training_time,
                'Validation Time': validation_time
            }
        )

    print("")
    print("Training complete!")


    # predictions, labels, metrics three files
    # content for every fold, every epoch, and every hyperparameter







    print("Total training took {:} (h:mm:ss)".format(format_time(time.time()-total_t0)))

In [33]:
wandb.agent(sweep_id,function=train)
data2.to_csv('500_balanced_true_RE_dev.csv')

wandb: Agent Starting Run: jvzi2azh with config:
wandb: 	batch_size: 16
wandb: 	epochs: 20
wandb: 	learning_rate: 1e-05
wandb: 	warmup_ratio: 0
wandb: Currently logged in as: nickmcneal (use `wandb login --relogin` to force relogin)


cuda


Some weights of the model checkpoint at microsoft/BiomedNLP-PubMedBERT-base-uncased-abstract were not used when initializing BertForSequenceClassification: ['cls.predictions.decoder.bias', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSeque

batch_size =  16
Learning_rate =  1e-05
epochs => 20

======== Epoch 1 / 20 ========
Training...

  Average training loss: 1.41
  Training epoch took: 0:00:05

Running Validation...
  Validation Loss: 1.26
  Validation took: 0:00:02
labels: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,


  Average training loss: 0.76
  Training epoch took: 0:00:04

Running Validation...
  Validation Loss: 0.81
  Validation took: 0:00:02
labels: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

...


  Average training loss: 0.03
  Training epoch took: 0:00:03

Running Validation...
  Validation Loss: 0.73
  Validation took: 0:00:02
labels: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 


  Average training loss: 0.02
  Training epoch took: 0:00:03

Running Validation...
  Validation Loss: 0.75
  Validation took: 0:00:02
labels: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 


  Average training loss: 0.02
  Training epoch took: 0:00:03

Running Validation...
  Validation Loss: 0.75
  Validation took: 0:00:02
labels: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

avg_train_loss,█▅▃▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
avg_val_loss,█▃▃▁▁▃▂▃▃▃▃▃▃▃▃▃▃▃▃▃
train_batch_loss,█▇▅▅▃▃▃▂▁▁▁▂▁▁▁▁▁▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
val_accuracy,▁▆▆██▇█▇██▇▇█▇▇█████
avg_train_loss,0.01859
avg_val_loss,0.76055
train_batch_loss,0.00221
val_accuracy,0.82996


wandb: Sweep Agent: Waiting for job.
wandb: Sweep Agent: Exiting.
